# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,79.83,69,67,7.63,PF,1611949118
1,Topi,34.0704,72.6236,48.00,58,0,6.02,PK,1611949118
2,Kapaa,22.0752,-159.3190,73.99,78,75,20.71,US,1611949118
3,Saskylakh,71.9167,114.0833,-38.90,79,79,2.04,RU,1611949118
4,Walvis Bay,-22.9575,14.5053,66.99,77,100,8.05,NaN,1611948624
...,...,...,...,...,...,...,...,...,...
550,Guaratinguetá,-22.8164,-45.1925,95.00,36,40,6.91,BR,1611949220
551,Zarubino,42.6300,131.0922,2.55,85,100,8.46,RU,1611949220
552,Namatanai,-3.6667,152.4333,80.98,84,68,11.16,PG,1611948698
553,Nānākuli,21.3906,-158.1547,73.40,73,75,8.05,US,1611949220


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
gmaps.configure(api_key = g_key)
locations = weather_df[['Lat', 'Lng']]
humidity = weather_df['Humidity']

In [36]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
ideal_con_df = weather_df.loc[(weather_df["Wind Speed"] <= 15) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 75) & (weather_df["Max Temp"] <= 80)].dropna()
ideal_con_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Saint-Philippe,-21.3585,55.7679,78.80,69,0,4.61,RE,1611949122
38,Pisco,-13.7000,-76.2167,77.00,65,0,6.91,PE,1611949013
93,Malindi,-3.2175,40.1191,77.70,85,0,11.32,KE,1611949135
187,Ilo,-17.6394,-71.3375,77.00,61,0,11.50,PE,1611949152
273,Xai-Xai,-25.0519,33.6442,76.30,89,0,3.85,MZ,1611949167
447,Korhogo,9.4500,-5.6333,79.70,16,0,2.82,CI,1611949196
467,Gayéri,12.6497,0.4931,78.42,16,0,9.78,BF,1611949200
506,Am Timan,11.0297,20.2827,78.84,18,0,5.59,TD,1611949211
537,Saint-Pierre,-21.3393,55.4781,78.80,69,0,4.61,RE,1611949217


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = ideal_con_df.loc[:,['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,Saint-Philippe,RE,-21.3585,55.7679,
38,Pisco,PE,-13.7000,-76.2167,
93,Malindi,KE,-3.2175,40.1191,
187,Ilo,PE,-17.6394,-71.3375,
273,Xai-Xai,MZ,-25.0519,33.6442,
447,Korhogo,CI,9.4500,-5.6333,
467,Gayéri,BF,12.6497,0.4931,
506,Am Timan,TD,11.0297,20.2827,
537,Saint-Pierre,RE,-21.3393,55.4781,


In [39]:
#base & params
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {'type' : 'lodging',
          'radius' : 5000,
          'key' : g_key,
}

#for loop for nearby
for idx, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['location'] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {idx}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
   
    #try/except
    try:
        hotel_df.loc[idx, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        


    except (KeyError, IndexError):
        print('Missing value...skipping.')
        
    print('_______________')


print('_______Finsihed_______')

Retrieving Results for Index 27: Saint-Philippe.
Closest hotel in Saint-Philippe is Chambres d'hôte "La Trinité".
_______________
Retrieving Results for Index 38: Pisco.
Closest hotel in Pisco is Hotel San Isidro Oficial.
_______________
Retrieving Results for Index 93: Malindi.
Closest hotel in Malindi is African House Resort.
_______________
Retrieving Results for Index 187: Ilo.
Closest hotel in Ilo is Torreblanca Hotel.
_______________
Retrieving Results for Index 273: Xai-Xai.
Closest hotel in Xai-Xai is Complexo da Familia.
_______________
Retrieving Results for Index 447: Korhogo.
Closest hotel in Korhogo is Hôtel Tchinbota.
_______________
Retrieving Results for Index 467: Gayéri.
Missing value...skipping.
_______________
Retrieving Results for Index 506: Am Timan.
Closest hotel in Am Timan is Palais du chef Canton Salamt.
_______________
Retrieving Results for Index 537: Saint-Pierre.
Closest hotel in Saint-Pierre is Lindsey Hôtel.
_______________
_______Finsihed_______


In [53]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,Saint-Philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
38,Pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial
93,Malindi,KE,-3.2175,40.1191,African House Resort
187,Ilo,PE,-17.6394,-71.3375,Torreblanca Hotel
273,Xai-Xai,MZ,-25.0519,33.6442,Complexo da Familia
447,Korhogo,CI,9.4500,-5.6333,Hôtel Tchinbota
467,Gayéri,BF,12.6497,0.4931,
506,Am Timan,TD,11.0297,20.2827,Palais du chef Canton Salamt
537,Saint-Pierre,RE,-21.3393,55.4781,Lindsey Hôtel


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))